### Case Study

In this case study, you will be working with the same Boston housing data you saw in the previous lesson.  However, you are now better equipped to work with the complexity that actually exists in this dataset.

First, let's get the libraries and data set up.

In [123]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from patsy import dmatrices
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

boston_data = load_boston()
df = pd.DataFrame()
df['MedianHomePrice'] = boston_data.target
df2 = pd.DataFrame(boston_data.data)
df2.columns = boston_data.feature_names
df = df.join(df2)
df.head()

,MedianHomePrice,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,21.6,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,34.7,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,33.4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,36.2,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


You will be using the **df** dataframe for this notebook.  You can find a description of the features in the dataset [here](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).  Specifically, you will be attempting to build the best model to predict the median home price using the other variables in the dataset. 

`1.` Use [train_test_split](http://scikit-learn.org/stable/modules/cross_validation.html), to create a training and testing dataset, where 20% of the data is in the test set.  Use a random state of 0. Store your results in `X_train, X_test, y_train, y_test`.

In [124]:
X_data = df.drop('MedianHomePrice', axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
                 X_data, df['MedianHomePrice'], test_size=0.2, random_state=0)

Unless specified, only use the `X_train` and `y_train` variables to answer the following questions.

`2.` To begin, obtain a summary of each of the features in the dataset.  Use the results of your summary to answer the first and second quizzes question below.  Also compare each variable to one another using the [corr](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) method.

In [125]:
df2 = pd.DataFrame()
df2['MedianHomePrice'] = y_train
df2[list(df.columns)[1:]] = X_train

In [126]:
df2.describe()

,MedianHomePrice,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,22.611881,3.361572,11.794554,11.123441,0.069307,0.555886,6.301797,69.027228,3.770242,9.403465,402.844059,18.462376,358.785446,12.706188
std,9.238777,8.130933,23.636906,6.993011,0.254290,0.115201,0.694692,28.106955,2.080583,8.661147,170.857022,2.194821,88.663438,7.299031
min,5.000000,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.174200,1.000000,187.000000,12.600000,0.320000,1.730000
25%,16.775000,0.078935,0.000000,5.130000,0.000000,0.449000,5.887750,45.675000,2.087875,4.000000,276.000000,17.400000,376.132500,6.727500
50%,21.400000,0.256510,0.000000,9.125000,0.000000,0.538000,6.211000,77.950000,3.190950,5.000000,322.000000,19.100000,391.600000,11.300000
75%,25.525000,3.202962,20.000000,18.100000,0.000000,0.631000,6.675000,93.900000,5.141475,24.000000,666.000000,20.200000,396.060000,17.112500
max,50.000000,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


In [127]:
df2.corr()

,MedianHomePrice,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
MedianHomePrice,1.000000,-0.407916,0.409824,-0.525054,0.177620,-0.459566,0.705039,-0.413340,0.281013,-0.425133,-0.511025,-0.563566,0.354226,-0.755666
CRIM,-0.407916,1.000000,-0.201704,0.391299,-0.058658,0.433184,-0.197405,0.338171,-0.370289,0.621224,0.574698,0.278578,-0.320797,0.460089
ZN,0.409824,-0.201704,1.000000,-0.534523,-0.024872,-0.527536,0.348199,-0.585812,0.656658,-0.313497,-0.329665,-0.396806,0.171123,-0.428584
INDUS,-0.525054,0.391299,-0.534523,1.000000,0.019945,0.757107,-0.405883,0.635978,-0.699502,0.573361,0.720073,0.390679,-0.342290,0.606026
CHAS,0.177620,-0.058658,-0.024872,0.019945,1.000000,0.032281,0.107340,0.048548,-0.071933,-0.010475,-0.048810,-0.107799,0.079502,-0.061047
NOX,-0.459566,0.433184,-0.527536,0.757107,0.032281,1.000000,-0.304220,0.743162,-0.774841,0.635081,0.693064,0.199779,-0.384622,0.596475
RM,0.705039,-0.197405,0.348199,-0.405883,0.107340,-0.304220,1.000000,-0.261999,0.204028,-0.197837,-0.290858,-0.379197,0.140592,-0.620118
AGE,-0.413340,0.338171,-0.585812,0.635978,0.048548,0.743162,-0.261999,1.000000,-0.754547,0.444137,0.504211,0.244094,-0.253652,0.612759
DIS,0.281013,-0.370289,0.656658,-0.699502,-0.071933,-0.774841,0.204028,-0.754547,1.000000,-0.479621,-0.536082,-0.203303,0.268693,-0.503193
RAD,-0.425133,0.621224,-0.313497,0.573361,-0.010475,0.635081,-0.197837,0.444137,-0.479621,1.000000,0.904438,0.447445,-0.446020,0.480207


`2.` Now, use [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) to scale all of the x-variables in the dataset.  Store the result in `X_scaled_train`. Create a pandas dataframe and store the scaled x-variables, as well as the training response.  Name the dataframe `training_data`.

In [128]:
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train)
X_scaled_train.columns = list(df.columns)[1:]
df3 = X_scaled_train.join(y_train.reset_index())

`3.` Now, fit a linear moedel using **all** of your scaled features to predict the response (median home price).  Don't forget to add an intercept.  Use the results of your linear model to answer quiz 3 below.

In [129]:
df3['intercept'] = 1
X_vars_full = df3.drop(['MedianHomePrice','index'] , axis=1, inplace=False)
lm = sm.OLS(df3['MedianHomePrice'], X_vars_full)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        MedianHomePrice   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     102.1
Date:                Sun, 26 Nov 2017   Prob (F-statistic):          9.99e-117
Time:                        02:11:03   Log-Likelihood:                -1171.5
No. Observations:                 404   AIC:                             2371.
Df Residuals:                     390   BIC:                             2427.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.9616      0.296     -3.246      0.001      -1.544      -0.379
ZN             1.0566      0.341      3.100      0.002       0.387       1.727
INDUS          0.0409      0.443      0.092      0.926      -0.830       0.912
CHAS           0.5949      0.229      2.596      0.010       0.144       1.045
NOX           -1.8597      0.485     -3.837      0.000      -2.813      -0.907
RM             2.5681      0.317      8.090      0.000       1.944       3.192
AGE           -0.0855      0.402     -0.213      0.832      -0.876       0.705
DIS           -2.8815      0.445     -6.480      0.000      -3.756      -2.007
RAD            2.1088      0.607      3.475      0.001       0.916       3.302
TAX           -1.8746      0.665     -2.818      0.005      -3.183      -0.567
PTRATIO       -2.2951      0.300     -7.643      0.000      -2.886      -1.705
B              0.7279      0.261      2.786      0.006       0.214       1.242
LSTAT         -3.5986      0.395     -9.108      0.000      -4.375      -2.822
intercept     22.6119      0.223    101.567      0.000      22.174      23.050
==============================================================================
Omnibus:                      141.470   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              628.301
Skew:                           1.470   Prob(JB):                    3.68e-137
Kurtosis:                       8.355   Cond. No.                         9.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

`4.` Now use the function below to calculate the vifs for every x_variable in the dataset.  Use the results of the function to answer the fourth quiz below.

In [130]:
def vif_calculator(df, response):
    '''
    INPUT:
    df - a dataframe holding the x and y-variables
    response - the column name of the response as a string
    OUTPUT:
    vif - a dataframe of the vifs
    '''
    df2 = df.drop(response, axis = 1, inplace=False)
    features = "+".join(df2.columns)
    y, X = dmatrices(response + ' ~' + features, df, return_type='dataframe')
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["features"] = X.columns
    vif = vif.round(1)
    return vif

In [131]:
vifs = vif_calculator(df3, 'MedianHomePrice')

/opt/conda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [132]:
vifs

,VIF Factor,features
0,0.0,Intercept
1,1.8,CRIM
2,2.4,ZN
3,4.0,INDUS
4,1.1,CHAS
5,4.7,NOX
6,2.0,RM
7,3.3,AGE
8,4.0,DIS
9,7.8,RAD


`5.` Based on the results of looking at the p-values and VIFs, you determine to remove `AGE`, `NOX`, and `TAX` to start.  Fit a new linear model with these removed (but still with an intercept).  Use the results from this linear model and the earlier model to answer quiz 5 below.

In [133]:
X_vars_red = df3.drop(['MedianHomePrice','index','AGE','NOX','TAX'] , axis=1, inplace=False)
lm = sm.OLS(df3['MedianHomePrice'], X_vars_red)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        MedianHomePrice   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     122.8
Date:                Sun, 26 Nov 2017   Prob (F-statistic):          2.71e-114
Time:                        02:11:08   Log-Likelihood:                -1184.8
No. Observations:                 404   AIC:                             2392.
Df Residuals:                     393   BIC:                             2436.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.8745      0.304     -2.874      0.004      -1.473      -0.276
ZN             0.9663      0.338      2.861      0.004       0.302       1.631
INDUS         -1.1035      0.381     -2.895      0.004      -1.853      -0.354
CHAS           0.6852      0.234      2.925      0.004       0.225       1.146
RM             2.6867      0.319      8.411      0.000       2.059       3.315
DIS           -2.1124      0.403     -5.246      0.000      -2.904      -1.321
RAD            0.1670      0.362      0.462      0.644      -0.544       0.878
PTRATIO       -1.9592      0.292     -6.705      0.000      -2.534      -1.385
B              0.7874      0.267      2.947      0.003       0.262       1.313
LSTAT         -3.8684      0.375    -10.302      0.000      -4.607      -3.130
intercept     22.6119      0.229     98.650      0.000      22.161      23.063
==============================================================================
Omnibus:                      141.386   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              640.171
Skew:                           1.462   Prob(JB):                    9.74e-140
Kurtosis:                       8.430   Cond. No.                         4.71
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

`6.` Give the results from the previous linear regression model, remove the `RAD` variable from the model.  Then double check that all your VIFs are below 4.  All of your variables should now show a significant linear relationship with the response, and there should have been no change to the Rsquared value from the previous model.

In [134]:
X_vars_red2 = df3.drop(['MedianHomePrice','index','AGE','NOX','TAX','RAD'] , axis=1, inplace=False)
lm = sm.OLS(df3['MedianHomePrice'], X_vars_red2)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        MedianHomePrice   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     136.6
Date:                Sun, 26 Nov 2017   Prob (F-statistic):          2.50e-115
Time:                        02:11:09   Log-Likelihood:                -1184.9
No. Observations:                 404   AIC:                             2390.
Df Residuals:                     394   BIC:                             2430.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.8123      0.273     -2.980      0.003      -1.348      -0.276
ZN             0.9811      0.336      2.920      0.004       0.321       1.642
INDUS         -1.0628      0.371     -2.869      0.004      -1.791      -0.334
CHAS           0.6905      0.234      2.955      0.003       0.231       1.150
RM             2.7053      0.317      8.546      0.000       2.083       3.328
DIS           -2.1343      0.399     -5.343      0.000      -2.920      -1.349
PTRATIO       -1.9158      0.276     -6.930      0.000      -2.459      -1.372
B              0.7565      0.258      2.927      0.004       0.248       1.265
LSTAT         -3.8629      0.375    -10.303      0.000      -4.600      -3.126
intercept     22.6119      0.229     98.748      0.000      22.162      23.062
==============================================================================
Omnibus:                      144.228   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              667.636
Skew:                           1.487   Prob(JB):                    1.06e-145
Kurtosis:                       8.552   Cond. No.                         4.42
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [135]:
df4 = df3.drop(['index','AGE','NOX','TAX','RAD'] , axis=1, inplace=False)
vifs = vif_calculator(df4, 'MedianHomePrice')
vifs

/opt/conda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,0.0,Intercept
1,1.4,CRIM
2,2.2,ZN
3,2.6,INDUS
4,1.0,CHAS
5,1.9,RM
6,3.0,DIS
7,1.5,PTRATIO
8,1.3,B
9,2.7,LSTAT


`6.` Because extending to the test data using linear models via statsmodels is a bit tedious, we will be using [sklearn to do so](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Below is an example of fitting the full model to our data.  

Note (as you can read in the documentation), the data are scaled by default using sklearn's linear model.  

I have created the remaing X matrices in the cell below the model.  Use these to refit and see which model performs best on the test set.  Use the results to answer the final quiz question below.

In [152]:
X_data = df.drop('MedianHomePrice', axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
                 X_data, df['MedianHomePrice'], test_size=0.2, random_state=0)


lm_full = LinearRegression()
lm_full.fit(X_train, y_train)
lm_full.score(X_test, y_test)

0.58920115191864575

In [154]:
X_train_red = X_train.drop(['AGE','NOX','TAX'] , axis=1, inplace=False)
X_test_red = X_test.drop(['AGE','NOX','TAX'] , axis=1, inplace=False)


X_train_red2 = X_train.drop(['AGE','NOX','TAX','RAD'] , axis=1, inplace=False)
X_test_red2 = X_test.drop(['AGE','NOX','TAX','RAD'] , axis=1, inplace=False)


In [156]:
lm_red = LinearRegression()
lm_red.fit(X_train_red, y_train)
print(lm_red.score(X_test_red, y_test))

lm_red2 = LinearRegression()
lm_red2.fit(X_train_red2, y_train)
print(lm_red2.score(X_test_red2, y_test))


0.548663728376
0.545224153736
